In [18]:
import os
import warnings
from pathlib import Path
import pandas as pd
import pyodbc
warnings.filterwarnings("ignore")
from sqlalchemy import create_engine
import os

In [13]:
## folder path containing 12 csv files. 
csv_folder = Path(r"C:\Users\admin\Downloads\csv_folder")

In [14]:
for csv_file in csv_folder.glob("*.csv"):
    df = pd.read_csv(csv_file)
    nulls = df.isnull().sum()
    total = nulls.sum()
    print(f"\nChecking: {csv_file.name}")
    if total == 0:
        print("No nulls found.")
    else:
        print(f"{total} null values found:")
        print(nulls[nulls > 0])


Checking: colors.csv
24 null values found:
y1    12
y2    12
dtype: int64

Checking: elements.csv
26382 null values found:
design_id    26382
dtype: int64

Checking: inventories.csv
No nulls found.

Checking: inventory_minifigs.csv
No nulls found.

Checking: inventory_parts.csv
6766 null values found:
img_url    6766
dtype: int64

Checking: inventory_sets.csv
No nulls found.

Checking: minifigs.csv
No nulls found.

Checking: parts.csv
No nulls found.

Checking: part_categories.csv
No nulls found.

Checking: part_relationships.csv
No nulls found.

Checking: sets.csv
No nulls found.

Checking: themes.csv
147 null values found:
parent_id    147
dtype: int64


In [15]:
for csv_file in csv_folder.glob("*.csv"):
    df = pd.read_csv(csv_file)
    nulls = df.isnull().sum()
    total = nulls.sum()
    print(f"\n{csv_file.name}")
    if total == 0:
        print("no nulls")
    else:
        print(f"nulls: {total}")
        print(nulls[nulls > 0])
        ## fill numeric columns with mean
        for col in df.select_dtypes(include=["float64", "int64"]):
            df[col].fillna(df[col].mean(), inplace=True)
        ## fill object columns with mode
        for col in df.select_dtypes(include=["object"]):
            mode = df[col].mode()
            if not mode.empty:
                df[col].fillna(mode[0], inplace=True)
        df.to_csv(csv_file, index=False)
        print("saved")


colors.csv
nulls: 24
y1    12
y2    12
dtype: int64
saved

elements.csv
nulls: 26382
design_id    26382
dtype: int64
saved

inventories.csv
no nulls

inventory_minifigs.csv
no nulls

inventory_parts.csv
nulls: 6766
img_url    6766
dtype: int64
saved

inventory_sets.csv
no nulls

minifigs.csv
no nulls

parts.csv
no nulls

part_categories.csv
no nulls

part_relationships.csv
no nulls

sets.csv
no nulls

themes.csv
nulls: 147
parent_id    147
dtype: int64
saved


In [17]:
## Now i Check again whether these null values have been filled mean, mode
for csv_file in csv_folder.glob("*.csv"):
    df = pd.read_csv(csv_file)
    nulls = df.isnull().sum()
    total = nulls.sum()
    print(f"\nchecking {csv_file.name}")
    if total == 0:
        print("no nulls")
    else:
        print(f"total nulls: {total}")
        print(nulls[nulls > 0])


checking colors.csv
no nulls

checking elements.csv
no nulls

checking inventories.csv
no nulls

checking inventory_minifigs.csv
no nulls

checking inventory_parts.csv
no nulls

checking inventory_sets.csv
no nulls

checking minifigs.csv
no nulls

checking parts.csv
no nulls

checking part_categories.csv
no nulls

checking part_relationships.csv
no nulls

checking sets.csv
no nulls

checking themes.csv
no nulls


## Method 1: Insert Directly and Create tables in DB by loading data

In [27]:
server = 'LENOVO\\DAJANASQL'

In [36]:

database = 'LEGO2' 

In [29]:
connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
    "&Encrypt=yes"
    "&TrustServerCertificate=yes"
)

In [30]:

engine = create_engine(connection_string)

In [31]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'Oracle in OraDB19Home1', 'MySQL ODBC 8.3 ANSI Driver', 'MySQL ODBC 8.3 Unicode Driver', 'ODBC Driver 17 for SQL Server']


In [32]:
for filename in os.listdir(csv_folder):
    if filename.endswith(".csv"):
        table = Path(filename).stem
        path = csv_folder / filename
        print(f"Inserting {filename} into {table}...")
        try:
            df = pd.read_csv(path)
            df.to_sql(table, con=engine, if_exists="fail", index=False)
            print(f"done with {table}")
        except Exception as e:
            print(f"failed on {table}: {e}")

Inserting colors.csv into colors...
done with colors
Inserting elements.csv into elements...
done with elements
Inserting inventories.csv into inventories...
done with inventories
Inserting inventory_minifigs.csv into inventory_minifigs...
done with inventory_minifigs
Inserting inventory_parts.csv into inventory_parts...
done with inventory_parts
Inserting inventory_sets.csv into inventory_sets...
done with inventory_sets
Inserting minifigs.csv into minifigs...
done with minifigs
Inserting parts.csv into parts...
done with parts
Inserting part_categories.csv into part_categories...
done with part_categories
Inserting part_relationships.csv into part_relationships...
done with part_relationships
Inserting sets.csv into sets...
done with sets
Inserting themes.csv into themes...
done with themes


## Method 2: Loading Data from csv Files into Database after having previously created tables with PKs Constraints

In [33]:
database = 'LEGO_Rebrickable'

In [34]:
connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
    "&Encrypt=yes"
    "&TrustServerCertificate=yes"
    "&timeout=120")

In [35]:
engine = create_engine(connection_string, fast_executemany=True)

In [1]:
for filename in os.listdir(csv_folder):
    if filename.endswith(".csv"):
        table = Path(filename).stem
        path = csv_folder / filename
        print(f"inserting {filename} into {table}...")
        try:
            df = pd.read_csv(path)
            df.to_sql(table, con=engine, if_exists="append", index=False, chunksize=1000)
            print(f"done with {table}")
        except Exception as e:
            print(f"couldn't insert into {table}: {e}")

C:\Users\admin\AppData\Local\Temp\ipykernel_27660\2384002670.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Inserting 'colors.csv' into table 'colors'...
✅ Done inserting into colors
Inserting 'elements.csv' into table 'elements'...
✅ Done inserting into elements
Inserting 'inventories.csv' into table 'inventories'...
✅ Done inserting into inventories
Inserting 'inventory_minifigs.csv' into table 'inventory_minifigs'...
✅ Done inserting into inventory_minifigs
Inserting 'inventory_parts.csv' into table 'inventory_parts'...
✅ Done inserting into inventory_parts
Inserting 'inventory_sets.csv' into table 'inventory_sets'...
✅ Done inserting into inventory_sets
Inserting 'minifigs.csv' into table 'minifigs'...
✅ Done inserting into minifigs
Inserting 'parts.csv' into table 'parts'...
✅ Done inserting into parts
Inserting 'part_categories.csv' into table 'part_categories'...
✅ Done inserting into part_categories
Inserting 'part_relationships.csv' into table 'part_relationships'...
✅ Done inserting into part_relationships
Inserting 'sets.csv' into table 'sets'...
✅ Done inserting into sets
Inserti